In [2]:
import pandas as pd

# Load the CSV file and skip the first row
df = pd.read_csv("FY25_Candidates database.csv")

# Drop unnamed columns
# df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# Display the first few rows of the cleaned dataset
print(df.head())

ModuleNotFoundError: No module named 'pandas'

In [ ]:
print(df.columns)

NameError: name 'df' is not defined

In [ ]:
df.info()


NameError: name 'df' is not defined

In [ ]:
print(df["Technonlogies/Skills known"].dtype)
print(df["Tech_points"].dtype)
print(df["Sports"].dtype)
print(df["Sports_Points"].dtype)
print(df["ExtraCurriculum"].dtype)
print(df["ExxCur_Points"].dtype)


NameError: name 'df' is not defined

In [ ]:
# Replace missing values in 'Graduation CGPA' with 0.0
df['Graduation CGPA'] = df['Graduation CGPA'].fillna(0.0)

NameError: name 'df' is not defined

In [ ]:
# Select relevant features for prediction
features = ['10th Board%', '12th Board%', 'Graduation Degree', 'Grad-Specialization', 'Technonlogies/Skills known']
X = df[features]
y = df['Graduation CGPA']

# Convert categorical variables to dummy variables
X = pd.get_dummies(X, drop_first=True)

# Display the first few rows of the processed data
print(X.head())
print(y.head())

NameError: name 'df' is not defined

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Check for missing values in X_train
print(X_train.isnull().sum())

# Check for missing values in X_test
print(X_test.isnull().sum())

10th Board%                                                                                                                                                                                                                     0
12th Board%                                                                                                                                                                                                                     1
Graduation Degree_B.E.                                                                                                                                                                                                          0
Graduation Degree_BE                                                                                                                                                                                                            0
Grad-Specialization_Information Technology                                                      

In [ ]:
# Fill missing values with 0
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on X_train and transform both X_train and X_test
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Initialize the model
model = RandomForestRegressor(random_state=42)

# Train the model
model.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = model.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 2.314656475133332
R^2 Score: -7.007960435227176


In [ ]:
import pickle

# Save the model and scaler
pickle.dump(model, open("Models/model.pkl", 'wb'))
pickle.dump(scaler, open("Models/scaler.pkl", 'wb'))

In [ ]:
# Load the model and scaler
model = pickle.load(open("Models/model.pkl", 'rb'))
scaler = pickle.load(open("Models/scaler.pkl", 'rb'))

# Example new data (replace with actual data)
new_data = pd.DataFrame({
    '10th Board%': [85],
    '12th Board%': [90],
    'Graduation Degree': ['B.E'],
    'Grad-Specialization': ['Information Technology'],
    'Technonlogies/Skills known': ['Python, SQL, Machine Learning']
})

# Convert categorical variables to dummy variables
new_data = pd.get_dummies(new_data, drop_first=True)

# Ensure the new data has the same columns as the training data
new_data = new_data.reindex(columns=X.columns, fill_value=0)

# Scale the new data
new_data_scaled = scaler.transform(new_data)

# Make a prediction
# predicted_cgpa = model.predict(new_data_scaled)
# print(f"Predicted Graduation CGPA: {predicted_cgpa[0]}")

In [ ]:
import sklearn
print(sklearn.__version__)

1.3.2


In [ ]:
# Replace NaN values in 'Technonlogies/Skills known' with an empty string
df["Technonlogies/Skills known"] = df["Technonlogies/Skills known"].fillna("")
df["Organization worked with (If Any) Eg: Accenture/KPMG"] = df["Organization worked with (If Any) Eg: Accenture/KPMG"].fillna("")

In [ ]:
df['Graduation CGPA'] = df['Graduation CGPA'].fillna(0.0)


In [ ]:
def unified_student_search(search_terms):
    """
    Search students who match ALL terms across skills/sports/extracurriculars 
    
    Parameters:
    - search_terms (str): Comma-separated terms (e.g. "python,hackathon,football")
    
    Returns:
    - DataFrame of students where EACH term appears in at least one column
    """
    # Clean data
    df_clean = df.copy()
    df_clean['Technonlogies/Skills known'] = df_clean['Technonlogies/Skills known'].fillna('').str.lower()
    df_clean['Sports'] = df_clean['Sports'].fillna('no').str.lower()
    df_clean['ExtraCurriculum'] = df_clean['ExtraCurriculum'].fillna('no').str.lower()
    
    # Process search terms
    terms = [term.strip().lower() for term in search_terms.split(",")]
    
    # Filter students where ALL terms appear in any of the three columns
    def matches_all_terms(row):
        combined_text = f"{row['Technonlogies/Skills known']} {row['Sports']} {row['ExtraCurriculum']}"
        return all(term in combined_text for term in terms)
    
    results = df_clean[df_clean.apply(matches_all_terms, axis=1)]
    
    return results[[
        "Candidate Name",
        "Technonlogies/Skills known", 
        "Sports",
        "ExtraCurriculum",
        "Graduation CGPA",
        "Primary Email ID (College)"
    ]]


In [ ]:
# Example usage:
search_input = "Library"  # Must appear in ANY of the 3 columns
matching_students = unified_student_search(search_input)

# Display results
print(f"Students matching ALL terms: {search_input}")
print("="*60)
if not matching_students.empty:
    for idx, student in matching_students.iterrows():
        print(f"\n{student['Candidate Name']} (CGPA: {student['Graduation CGPA']})")
        print(f"Email: {student['Primary Email ID (College)']}")
        
        # Highlight which column contains each term
        for term in search_input.split(","):
            term = term.strip().lower()
            found_in = []
            if term in student['Technonlogies/Skills known'].lower():
                found_in.append("Skills")
            if term in student['Sports'].lower():
                found_in.append("Sports")
            if term in student['ExtraCurriculum'].lower():
                found_in.append("Extracurriculars")
            print(f"- '{term}' found in: {', '.join(found_in)}")
        
        print("="*60)
else:
    print("No students match ALL search terms")


Students matching ALL terms: Library

Anshika Gupta (CGPA: 8.76)
Email: anshika6304@gmail.com
- 'library' found in: Extracurriculars
